<a href="https://colab.research.google.com/github/carmenmiravete/barley_variant_calling/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 3 - Group Practice


## 1. Installation of the libraries.

**You need to add the installation of the library to use CUDA**

In [1]:
!pip install bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.3 MB/s eta 0:00:00


## 2. Import the packets that are required.

**You need to incorporate the packets to use CuPy**

In [2]:
import numpy as np
from Bio import SeqIO
import time


## 3. Definition of the constants and functions to work with DNA letters as numbers.

**Don't modify this part of the code**

In [3]:
DNA_A = 1
DNA_C = 2
DNA_T = 3
DNA_G = 4
DNA_NULL = 0
DNA_LETTERS = ['-', 'A', 'C', 'T', 'G']

def nucleotid2int(character):
  if character == "A":
    return DNA_A
  elif character == "C":
    return DNA_C
  elif character == "T":
    return DNA_T
  elif character == "G":
    return DNA_G
  else:
    return DNA_NULL

def dna2int(dna):
  return [nucleotid2int(character) for character in dna.upper()]

def int2dna(values):
  return [DNA_LETTERS[value] for value in values]

## 4. Definition of the constants to calculate the alignment

**Don't modify this part of the code**

In [4]:
POINTS_EQUAL = 1
POINTS_NON_EQUAL = -1
POINTS_GAP = -1

MOVEMENT_START = 0
MOVEMENT_RIGHT = 1
MOVEMENT_DIAG = 2
MOVEMENT_DOWN = 3

## 5. Function to fill the first column of the scores matrix and the movements matrix.
Convert it into a kernel

In [5]:
import cupy as cp
from cupyx import jit

In [6]:
@jit.rawkernel()
def fill_first_column_kernel(scores, movements, n_rows):
    row = jit.blockIdx.x
    if row < n_rows:
        scores[row, 0] = POINTS_GAP * row
        movements[row, 0] = MOVEMENT_DOWN

def fill_first_column(scores, movements, n_rows):
    num_blocks = n_rows
    threads_per_block = 1
    fill_first_column_kernel[num_blocks, threads_per_block](scores, movements, n_rows)


/usr/local/lib/python3.10/dist-packages/cupyx/jit/_interface.py:171: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


## 6. Function to fill the first row of the scores matrix and the movements matrix.

Convert it into a kernel



In [7]:
@jit.rawkernel()
def fill_first_row_kernel(scores, movements, n_cols):
    col = jit.blockIdx.x
    if col < n_cols:
        scores[0, col] = POINTS_GAP * col
        movements[0, col] = MOVEMENT_RIGHT

def fill_first_row(scores, movements, n_cols):
    num_blocks = n_cols
    threads_per_block = 1
    fill_first_row_kernel[num_blocks, threads_per_block](scores, movements, n_cols)


## 7. Functions to calculate the scores and the movements.

Convert `calculate_scores_antidiag` into a kernel and modify `calculate_scores` to call the kernel

In [8]:
@jit.rawkernel()
def calculate_scores_antidiag_kernel(sequence1, sequence2, scores, movements, antidiag, init_row, end_row, POINTS_EQUAL, POINTS_NON_EQUAL, POINTS_GAP, MOVEMENT_DIAG, MOVEMENT_DOWN, MOVEMENT_RIGHT, n_cols):
    row = jit.blockIdx.x + init_row
    if row <= end_row:
        col = antidiag - row
        comparison = POINTS_EQUAL * (sequence1[row - 1] == sequence2[col - 1]) + POINTS_NON_EQUAL * (sequence1[row - 1] != sequence2[col - 1])

        new_score_diag = scores[row-1, col-1] + comparison
        new_score_right = scores[row, col - 1] + POINTS_GAP
        new_score_down = scores[row - 1, col] + POINTS_GAP

        if (new_score_diag >= new_score_down) & (new_score_diag >= new_score_right):
            scores[row, col] = new_score_diag
            movements[row, col] = MOVEMENT_DIAG
        elif (new_score_down >= new_score_right):
            scores[row, col] = new_score_down
            movements[row, col] = MOVEMENT_DOWN
        else:
            scores[row, col] = new_score_right
            movements[row, col] = MOVEMENT_RIGHT

def calculate_scores_antidiag(seq1, seq2, scores, movements, n_rows, n_cols):
    num_blocks_x = (n_rows + 31) // 32
    num_blocks_y = (n_cols + 31) // 32
    threads_per_block_x = 32
    threads_per_block_y = 32

    calculate_scores_antidiag_kernel[(num_blocks_x, num_blocks_y), (threads_per_block_x, threads_per_block_y)](seq1, seq2, scores, movements, n_rows, n_cols, POINTS_EQUAL, POINTS_NON_EQUAL, POINTS_GAP, MOVEMENT_DIAG, MOVEMENT_DOWN, MOVEMENT_RIGHT)


In [9]:

def calculate_scores(seq1, seq2, scores, movements, n_rows, n_cols):
  for antidiag in range(2, n_rows+n_cols-1):
    if antidiag <= n_cols:
      init_row = 1
    else:
      init_row = antidiag - n_cols + 1

    if antidiag <= n_rows:
      end_row = antidiag - 1
    else:
      end_row = n_rows - 1

    n_rows_to_analyze = end_row - init_row + 1

    calculate_scores_antidiag(seq1, seq2, scores, movements, antidiag, init_row, end_row)

## 8. Function to calculate the results of the alignment

**Do not modify**

In [10]:
def fill_responses(seq1, seq2, scores, movements, n_rows, n_cols):
  col = n_cols - 1
  row = n_rows - 1

  result1 = ""
  result2 = ""
  while (col + row) > 0:
    if movements[row, col] == MOVEMENT_DIAG:
      if (row > 0) & (row < n_rows):
        result1 = seq1[row - 1] + result1
      else:
        result1 = '-' + result1

      if (col > 0) & (col < n_cols):
        result2 = seq2[col - 1] + result2
      else:
        result2 = '-' + result2

      row -= 1
      col -= 1

    elif movements[row, col] == MOVEMENT_RIGHT:
      result1 = '-' + result1

      if (col > 0) & (col < n_cols):
        result2 = seq2[col - 1] + result2
      else:
        result2 = '-' + result2

      col -= 1

    elif movements[row, col] == MOVEMENT_DOWN:
      if (row > 0) & (row < n_rows):
        result1 = seq1[row - 1] + result1
      else:
        result1 = '-' + result1

      result2 = '-' + result2
      row -= 1

    else:
      row -= 1
      col -= 1

  return result1, result2


## 9. Function to calculate the alignment.

You need to modify it to:
 * Send the sequences to the GPU
 * Run the kernels `fill_first_column`and `fill_first_row`
 * Get the matrix of the `scores` and `movements into the CPU

In [11]:
# Main function to run the alignment
def get_alignment(seq1, seq2):
    seq1_int = dna2int(seq1)
    seq1_gpu = cp.array(seq1_int, dtype=cp.int32)

    seq2_int = dna2int(seq2)
    seq2_gpu = cp.array(seq2_int, dtype=cp.int32)

    n_rows = len(seq1) + 1
    n_cols = len(seq2) + 1

    scores_gpu = cp.empty((n_rows, n_cols), dtype=cp.int32)
    movements_gpu = cp.empty((n_rows, n_cols), dtype=cp.int32)

    # Launch kernels
    fill_first_column_kernel[1, n_rows](scores_gpu, movements_gpu, n_rows)
    fill_first_row_kernel[1, n_cols](scores_gpu, movements_gpu, n_cols)
    for antidiag in range(2, n_rows + n_cols - 1):
        if antidiag <= n_cols:
            init_row = 1
        else:
            init_row = antidiag - n_cols + 1

        if antidiag <= n_rows:
            end_row = antidiag - 1
        else:
            end_row = n_rows - 1

        n_rows_to_analyze = end_row - init_row + 1

        calculate_scores_antidiag_kernel[(n_rows_to_analyze,), (1,)](seq1_gpu, seq2_gpu, scores_gpu, movements_gpu, antidiag, init_row, end_row, POINTS_EQUAL, POINTS_NON_EQUAL, POINTS_GAP, MOVEMENT_DIAG, MOVEMENT_DOWN, MOVEMENT_RIGHT, n_cols)

    # Fetch results from GPU
    scores_cpu = cp.asnumpy(scores_gpu)
    movements_cpu = cp.asnumpy(movements_gpu)

    result1, result2 = fill_responses(seq1, seq2, scores_cpu, movements_cpu, n_rows, n_cols)
    return scores_cpu[n_rows-1, n_cols-1], result1, result2


## 10. Main function to run the alignment.

You need to uplad the files of the first practice to test them.

In [12]:
seq1_file = "sequence_0_1.fasta"
seq1 = list(SeqIO.parse(open(seq1_file), "fasta"))[0].seq
seq2_file = "sequence_0_2.fasta"
seq2 = list(SeqIO.parse(open(seq2_file), "fasta"))[0].seq

print("Sequences")
print("Sequence 1:", seq1)
print("Sequence 2:", seq2)

start = time.time()

points, result1, result2 = get_alignment(seq1, seq2)

end = time.time()
print("\nTime for the alignment:", end - start, 's')

print("\nAlignment - Points ", points)
print("Sequence 1:", result1)
print("Sequence 2:", result2)


Sequences
Sequence 1: AATGAATGAATGAATCAATGAATGAAT
Sequence 2: AATAATAATAATGAATAATAAT

Time for the alignment: 1.0399177074432373 s

Alignment - Points  15
Sequence 1: AATGAATGAATGAATCAATGAATGAAT
Sequence 2: AAT-AAT-AAT-AATGAAT-AAT-AAT
